<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/TreePedia_Step3_getting_gsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To access the google streetview API, one should have the API key.**
* First, get started with Google Maps platform: https://developers.google.com/maps/gmp-get-started
* Then, get an API key to use Street View Static API: https://developers.google.com/maps/documentation/streetview/get-api-key

---
This notebook uses google-streetview package: https://pypi.org/project/google-streetview/. 
* For documentation, please see https://rrwen.github.io/google_streetview/



In [18]:
import pandas as pd
import math

import os
import requests
import shutil
from glob import glob

try:
  from urllib.parse import urlencode
except ImportError:
  from urllib import urlencode

from IPython.display import Image, display

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/My\ Drive/StreetPsych

Chicago  HydePark  key.txt


In [4]:
# Google Street View Static API key
with open('drive/My Drive/StreetPsych/key.txt') as file:
  api_key = file.read()

In [67]:
#location = 'HydePark'
#location = 'Chicago'
location = 'Brookline_MA'

picked_df = pd.read_csv('drive/My Drive/StreetPsych/' + location + '/gsv_seed_' + location + '.csv')
picked_df

,Unnamed: 0,lat,lng,bearing,osm_index,gsv_lat,gsv_lng,pano_id,date,status
0,0,42.327738,-71.125754,233.914886,0,42.327726,-71.125726,qPFd8ZiCTDtFRbIuXrD_fQ,2018-06,OK
1,1,42.308979,-71.147382,111.039266,1,NaN,NaN,NaN,NaN,NaN
2,2,42.308377,-71.146599,333.033300,3,NaN,NaN,NaN,NaN,NaN
3,3,42.312865,-71.164519,285.269836,21,42.312834,-71.164465,_6Zd2vQL1HJBJ0Rc3yvxNA,2011-07,OK
4,4,42.313030,-71.165028,303.632335,21,42.312999,-71.164982,ACStUzQrXsKtXm-CR17cTw,2011-07,OK
...,...,...,...,...,...,...,...,...,...,...
1405,1405,42.332801,-71.115996,60.352672,3264,42.332802,-71.116007,Dh9jLDcWVcrIbrI5eGkneA,2018-05,OK
1406,1406,42.333009,-71.115501,60.352591,3264,42.333028,-71.115462,DUW77JTp8KB6pLnvqGPYdA,2018-05,OK
1407,1407,42.300896,-71.150459,259.672377,3265,42.300895,-71.150479,IGjcwamebot1aOxpFalugg,2016-06,OK
1408,1408,42.332130,-71.125745,117.124299,3304,42.332129,-71.125785,8oBKpVMlNAy9dGh9h0azqg,2019-08,OK


In [6]:
(int(picked_df.iloc[0]['bearing']) + 90) % 360

186

In [7]:
!mkdir tmp

In [68]:
def download_gsv(api_key, pano_id, heading, image_dir):
  
  # construct the GSV api link
  params = {
    'key': api_key,
    'pano': pano_id,
    'heading': str(heading),
    'fov': '90',
    'size': '630x630' 
  }
  site_api='https://maps.googleapis.com/maps/api/streetview'
  url = site_api + '?' + urlencode(params)
  file_name = image_dir + '/' + pano_id + '_HEADING_' + str(heading) + '.jpg'

  if os.path.exists(image_dir) == False:
    os.mkdir(image_dir)

  # download and save the GSV image
  r = requests.get(url, stream=True)
  if r.status_code == 200: # if request is successful
    with open(file_name, 'wb') as f:
      r.raw.decode_content = True
      shutil.copyfileobj(r.raw, f)


idx = 100
pano_id = picked_df.iloc[idx]['pano_id']
heading = (int(picked_df.iloc[idx]['bearing']) + 90) % 360

download_gsv(api_key, pano_id, heading, 'tmp')

#display(Image('tmp.jpg'))
!ls tmp
!rm tmp/*

R2-0gk7jC-qKIZWcE7U-Ew_HEADING_94.jpg


In [69]:
# the directory to save GSV images
image_dir = 'drive/My Drive/StreetPsych/' + location + '/images'

if os.path.exists(image_dir) == False:
  os.mkdir(image_dir)

In [70]:
for idx, row in picked_df.iterrows():

  if pd.isna(row['date']):
    continue
  
  if (row['date'] > '2018'):
    pano_id = row['pano_id']
    heading1 = (int(row['bearing']) + 90) % 360
    download_gsv(api_key, pano_id, heading1, image_dir)
    heading2 = (int(row['bearing']) + 270) % 360
    download_gsv(api_key, pano_id, heading2, image_dir)

  if idx % 300 == 299:
    print('Going through ', idx)

print('Done.')

#pano_id = picked_df.iloc[idx]['pano_id']
#heading = (int(picked_df.iloc[idx]['bearing']) + 90) % 360
#download_gsv(api_key, pano_id, heading, 'tmp.jpg')

print(glob(image_dir + '/*'))

Going through  299
Going through  599
Going through  899
Going through  1199
Done.
['drive/My Drive/StreetPsych/Brookline_MA/images/qPFd8ZiCTDtFRbIuXrD_fQ_HEADING_323.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/qPFd8ZiCTDtFRbIuXrD_fQ_HEADING_143.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/Q8Hhd4tDyZCjHn_QWPlxaQ_HEADING_296.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/Q8Hhd4tDyZCjHn_QWPlxaQ_HEADING_116.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/_Ldkbi2qAwBqQbwFWzAdKA_HEADING_288.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/_Ldkbi2qAwBqQbwFWzAdKA_HEADING_108.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/clz6lbx43ExqASyDio795A_HEADING_288.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/clz6lbx43ExqASyDio795A_HEADING_108.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/sexyndnHpDGEx_LNkQmpkA_HEADING_267.jpg', 'drive/My Drive/StreetPsych/Brookline_MA/images/sexyndnHpDGEx_LNkQmpkA_HEADING_87.jpg', 'drive/My Drive/StreetPsych

In [64]:
#row['date'] > '2019'
#~math.isnan(int(row['date']))
#pd.isna(row['date'])
#pd.isna(row['date'])
#(row['date'] > '2019')
idx

386

In [33]:
shutil.rmtree(image_dir)
print(glob(image_dir + '/*'))

[]
